In [1]:
# Initializing Spark
with open("setupPySpark.py", "r") as setup_file:
    exec(setup_file.read())


In [2]:
# Spark context
from pyspark.sql.session import SparkSession

spark = SparkSession(sc)

In [3]:
# SQL context
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [11]:
# Importing required functions
from util import read_file, read_folder, get_character_similarity
from pandas import Series, DataFrame
from util_spark import remove_stopwords_spark, detect_language_spark, flatten_list_of_tokens, spell_correct_tokens_spark, get_semantic_similarity_spark
from tokenization_spark import tokenize_sentence_nltk_spark
from pyspark.sql.functions import col
from pos_tagging_spark import run_treetagger_pos_tag_spark
from modeling_spark import run_word2vec_model_pyspark
from json import load

In [12]:
# Reading input file(s) using python's default libraries
in_file = load(open("in_file.cfg"))
patterns_file = in_file["patterns_file"]
file_folder = in_file["file_folder"]
label = in_file["label"]
column = in_file["column"]
in_type = in_file["in_type"]
in_file = in_file["in_file"]
if file_folder == "file":
    strings = read_file(in_file, in_type = in_type)
    if in_type == "text":
        strings = tokenize_sentence_nltk(strings)
        strings = DataFrame(strings)[0]
    elif in_type == "html_chat":
        timestamp = strings[2]
        meta_data = strings[1]
        strings = strings[0]
        strings[label] = meta_data["Comment"]
        labels = strings[label]
        strings = strings[column]
    else:
        if label in strings.columns:
            labels = strings[label]
        strings = strings[column]
else:
    strings = read_folder(in_file, in_type = in_type)
    patterns = Series([".*" + x + ".*" for x in open(patterns_file, 'r').readlines()])

In [6]:
# Appending conversation together and creating spark data frome
try:
    strings['conversation'] = strings['conversation'].apply(lambda x: ". ".join(x["Message"]))
except:
    pass
sentenceDataFrame = spark.createDataFrame(strings)

In [7]:
# Creating list of sentences for each conversation
sentenceDataFrame = tokenize_sentence_nltk_spark(df = sentenceDataFrame, in_col = "conversation")

In [8]:
# Language identification and filtering
sentenceDataFrame = detect_language_spark(df = sentenceDataFrame, in_col = "conversation", out_col = "language")
sentenceDataFrame = sentenceDataFrame.where(col('language') == "en")

In [9]:
# POS tagging and lemmatization using TreeTagger
sentenceDataFrame = run_treetagger_pos_tag_spark(df = sentenceDataFrame, in_col = "conversation", out_col = "pos", get_lemma = True)

In [10]:
# Merging 2 consecutive words if a) Words are incorrectly spelled and b) Merged word is correctly spelled
sentenceDataFrame = spell_correct_tokens_spark(df = sentenceDataFrame, in_col = "pos")

In [11]:
# Flattening out token of rows and running word2vec model
sentenceDataFrame = flatten_list_of_tokens(sentenceDataFrame, in_col = "pos")
model, sentenceDataFrame = run_word2vec_model_pyspark(sentenceDataFrame, in_col = "pos", vec_size = 100, in_type = "tokens", out_col = "result")

In [12]:
# Collecting document vectors in a list
doc_vecs = []
for row in sentenceDataFrame.select('result').collect():
    doc_vecs = doc_vecs + [row['result']]

In [13]:
sim1 = get_semantic_similarity_spark(model)

In [14]:
sim1.head()

,for,this,in,move,have,your,point,guy,b2b,agree,...,no,we,let,ramification,when,away,and,today,the,cant
for,1.000000,-0.105910,0.000170,-0.069652,0.040336,0.069056,0.092621,0.056477,0.079010,0.165845,...,-0.022835,0.008508,-0.000052,0.074926,-0.033127,-0.054825,-0.124980,0.152799,0.007628,0.017553
this,-0.105910,1.000000,-0.069082,0.003931,0.060735,0.146523,-0.044929,0.096866,0.305122,-0.158147,...,-0.064297,0.052718,-0.028738,-0.034106,0.012105,0.140868,-0.031277,0.017369,-0.196300,-0.056384
in,0.000170,-0.069082,1.000000,-0.037609,-0.059549,0.070181,0.036973,-0.034291,-0.063641,-0.062104,...,-0.034954,0.124539,-0.146022,0.064458,-0.079840,0.228045,-0.057792,-0.202181,0.109813,0.019643
move,-0.069652,0.003931,-0.037609,1.000000,-0.126582,-0.048928,0.046263,0.021864,0.034268,-0.074104,...,0.036902,0.004310,0.156517,0.016187,-0.010523,0.028652,0.049523,-0.021243,-0.058675,-0.041668
have,0.040336,0.060735,-0.059549,-0.126582,1.000000,-0.037032,-0.220678,-0.119094,0.068659,-0.055997,...,-0.105475,0.043235,-0.009960,0.138637,-0.109652,0.012543,-0.024905,0.033226,-0.038240,-0.002423


In [15]:
ratio = get_character_similarity(sim1.columns, ratio_type = 'ratio')

In [16]:
partial_ratio = get_character_similarity(sim1.columns, ratio_type = 'partial_ratio')

In [17]:
token_sort_ratio = get_character_similarity(sim1.columns, ratio_type = 'token_sort_ratio')

In [18]:
token_set_ratio = get_character_similarity(sim1.columns, ratio_type = 'token_set_ratio')

In [19]:
sim1 = sim1[ratio.columns]
sim1 = sim1.loc[ratio.columns]

In [20]:
semantic_weight = 0.5
ratio_weight = 0.4
partial_ratio_weight = 0.4
token_sort_ratio_weight = 0.1
sim = 1 - (semantic_weight*sim1 + (ratio_weight*ratio + partial_ratio_weight*partial_ratio + token_sort_ratio_weight*token_sort_ratio + (1-ratio_weight-partial_ratio_weight-token_sort_ratio_weight)*token_set_ratio)*(1-semantic_weight))